In [1]:
import pandas as pd
from fastdata.core import *
def euclidean_distance(df1, df2):
    import numpy as np
    eucl_d = np.linalg.norm(df1.values - df2.values, axis=1)
    return pd.DataFrame({'eucl_d':eucl_d}, index=df1.index).reset_index()
import plotly.express as px
import pandas as pd
from fastdata.core import *

# Steps to Clean the Data
## 1. Exporting the FIFA 2020 file

In [2]:
data = pd.read_csv(
    sep=",", 
    decimal=".", 
    filepath_or_buffer="players_20.csv")

## 2. Dropping extra columns we don't need 

In [3]:
data = data.drop(
    columns=["player_url","long_name","nationality","real_face","player_tags","team_jersey_number","loaned_from","joined","contract_valid_until","nation_position","nation_jersey_number"
            , "dob","player_positions","sofifa_id","release_clause_eur","value_eur", "player_traits",
            "gk_diving","gk_handling","gk_kicking","gk_reflexes","gk_speed","gk_positioning"])

## 3. Cleaning the "Body Type" Column

In [4]:
data["standard_bodytype"] = data.eval("body_type in ['Normal', 'Lean', 'Stocky']", engine="python")

In [5]:
data["body_type_clean"] = data["body_type"].fdt.replace_based_on_condition(
    when=False, 
    cond=data["standard_bodytype"], 
    replace_with="Normal")

## 4. Convert "text" variables to "dummy variables"

In [6]:
data = pd.get_dummies(
    dummy_na=False, 
    data=data, 
    columns=["preferred_foot","body_type_clean","work_rate"])

In [7]:
data = data.drop(
    columns=["body_type","standard_bodytype"])

## 5. Drop "NA" values

In [8]:
data = data.dropna(
    axis="index", 
    how="any")

## 6. Remove unwanted characters "+/-"

In [9]:
data = data.fdt.clean_text_columns(
    keep_unmatched=True, 
    columns=['ls', 'st', 'rs',
       'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',
       'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
       'rcb', 'rb'], 
    mode="before_character", 
    character="-")

/Users/peremolinsayuso/opt/anaconda3/envs/dev_react_jupyterlab/lib/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [10]:
data = data.fdt.clean_text_columns(
    keep_unmatched=True, 
    columns=['ls', 'st', 'rs',
       'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',
       'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
       'rcb', 'rb'], 
    mode="before_character", 
    character="+")

## 7. Choose your player (in our case, "Messi") 

In [11]:
messi = data.query("short_name=='L. Messi'", engine="python").copy()

## 8. <font color='red'>Filter for your parameters: </font>
### a. First, only want Messi's positions, Right Winger "RW" 
### b. Next, we want players whose wages are less than 100K / week
### c. Finally, we want players who are 25 or younger


In [12]:
data = data.query("(team_position in ['RW'] and wage_eur < 200000 and age <= 25)", engine="python").copy()

## 9. Remove unwanted cells

In [13]:
data = data.drop(
    columns=["club","team_position", "wage_eur"])

In [14]:
data = data.set_index('short_name')

In [15]:
messi = messi.drop(
    columns=["short_name","club","team_position", "wage_eur"])

## We now have two datasets:
    One Vector with Messi's characteristics 
    One dataset with all Right Wingers

## 10. Convert all cells to numeric cells (we will need this to calculate the Euclidean distance)

In [16]:
data = data.apply(
    func=pd.to_numeric)

In [17]:
messi = messi.apply(
    func=pd.to_numeric)

## 11. Run the Euclidean Distance function

In [18]:
result = euclidean_distance(data,
    df2=messi)

## 12. Get your result! 

Sort by distance

In [19]:
result = result.sort_values(
    by="eucl_d", 
    ascending=True)

Get top 20

In [20]:
top_twenty = result.head(20)

In [23]:
px.bar(top_twenty,
    x="short_name", 
    y="eucl_d")